In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models
import timm
import cupy

In [2]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
print(f'Using device: {device}')

Using device: cuda


In [4]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
# Define path to your training data
train_data_path = 'AugmentedAlzheimerDataset'

# Load dataset using ImageFolder and split into train and validation
train_dataset = ImageFolder(root=train_data_path, transform=transform)

In [6]:
# Assuming you want to split 80% for training and 20% for validation
train_size = int(0.8 * len(train_dataset))
val_size = int(0.1 * len(train_dataset))
test_size = len(train_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(train_dataset, [train_size, val_size, test_size])

In [7]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32,shuffle=False)

In [8]:
# Load GhostNet model from PyTorch Hub
ghostnet = torch.hub.load('huawei-noah/ghostnet', 'ghostnet_1x', pretrained=True)

Using cache found in C:\Users\sushant mittal/.cache\torch\hub\huawei-noah_ghostnet_master


In [19]:
# Modify the model (adding custom layers)
class CustomGhostNet(nn.Module):
    def __init__(self, original_model, num_classes=4):  # Assuming 4 classes
        super(CustomGhostNet, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-1])  # Remove the last layer
        self.custom_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Dropout(0.2),  # Adjusted dropout rate
            nn.Linear(512,128),
            nn.ReLU(),
            nn.Linear(128,32),
            nn.ReLU(),
            nn.Linear(32,num_classes),
            nn.Softmax(dim=1)  # Apply softmax along the class dimension
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.custom_layers(x)
        return x
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomGhostNet(ghostnet, num_classes=4).to(device)

In [20]:
def replace_activations(model, old_activation, new_activation):
    for name, module in model.named_modules():
        for child_name, child in module.named_children():
            if isinstance(child, old_activation):
                setattr(module, child_name, new_activation())
    return model

# Replace ReLU with LeakyReLU in the entire model
model = replace_activations(model, nn.ReLU, nn.PReLU)

model = model.to(device)

In [21]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # SGD with momentum

In [22]:
# Training loop
num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    train_acc = correct_train / total_train * 100
    
    # Validation loop
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss = criterion(outputs, labels)
            
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            
            val_running_loss += val_loss.item()
    
    val_epoch_loss = val_running_loss / len(val_loader)
    val_acc = correct_val / total_val * 100
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {epoch_loss:.4f}, '
          f'Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_epoch_loss:.4f}, '
          f'Val Acc: {val_acc:.2f}%')


Epoch [1/15], Train Loss: 1.3485, Train Acc: 34.30%, Val Loss: 1.2838, Val Acc: 40.85%
Epoch [2/15], Train Loss: 1.1756, Train Acc: 54.88%, Val Loss: 1.1068, Val Acc: 61.65%
Epoch [3/15], Train Loss: 1.0778, Train Acc: 65.52%, Val Loss: 1.0530, Val Acc: 67.83%
Epoch [4/15], Train Loss: 1.0312, Train Acc: 70.45%, Val Loss: 1.0084, Val Acc: 73.22%
Epoch [5/15], Train Loss: 0.9919, Train Acc: 74.65%, Val Loss: 0.9532, Val Acc: 78.81%
Epoch [6/15], Train Loss: 0.9539, Train Acc: 78.78%, Val Loss: 0.9292, Val Acc: 81.14%
Epoch [7/15], Train Loss: 0.9249, Train Acc: 81.72%, Val Loss: 0.9085, Val Acc: 83.61%
Epoch [8/15], Train Loss: 0.8959, Train Acc: 84.75%, Val Loss: 0.8904, Val Acc: 85.05%
Epoch [9/15], Train Loss: 0.8754, Train Acc: 86.76%, Val Loss: 0.8686, Val Acc: 87.73%
Epoch [10/15], Train Loss: 0.8564, Train Acc: 88.77%, Val Loss: 0.8740, Val Acc: 86.70%
Epoch [11/15], Train Loss: 0.8413, Train Acc: 90.27%, Val Loss: 0.8686, Val Acc: 87.11%
Epoch [12/15], Train Loss: 0.8315, Train 

In [23]:
from torchsummary import summary
summary(model, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 112, 112]             432
       BatchNorm2d-2         [-1, 16, 112, 112]              32
             PReLU-3         [-1, 16, 112, 112]               1
            Conv2d-4          [-1, 8, 112, 112]             128
       BatchNorm2d-5          [-1, 8, 112, 112]              16
             PReLU-6          [-1, 8, 112, 112]               1
            Conv2d-7          [-1, 8, 112, 112]              72
       BatchNorm2d-8          [-1, 8, 112, 112]              16
             PReLU-9          [-1, 8, 112, 112]               1
      GhostModule-10         [-1, 16, 112, 112]               0
           Conv2d-11          [-1, 8, 112, 112]             128
      BatchNorm2d-12          [-1, 8, 112, 112]              16
           Conv2d-13          [-1, 8, 112, 112]              72
      BatchNorm2d-14          [-1, 8, 1

In [24]:
# Evaluate the model on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total * 100
print(f'Final Testing Accuracy: {test_accuracy:.2f}%')

Final Testing Accuracy: 93.47%


In [25]:
# Save the trained model
torch.save(model.state_dict(), 'ghostnet_softmax_15epochs_93point47_test_acc.pth')

In [25]:
from torchsummary import summary
summary(model, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 112, 112]             432
       BatchNorm2d-2         [-1, 16, 112, 112]              32
              ReLU-3         [-1, 16, 112, 112]               0
            Conv2d-4          [-1, 8, 112, 112]             128
       BatchNorm2d-5          [-1, 8, 112, 112]              16
              ReLU-6          [-1, 8, 112, 112]               0
            Conv2d-7          [-1, 8, 112, 112]              72
       BatchNorm2d-8          [-1, 8, 112, 112]              16
              ReLU-9          [-1, 8, 112, 112]               0
      GhostModule-10         [-1, 16, 112, 112]               0
           Conv2d-11          [-1, 8, 112, 112]             128
      BatchNorm2d-12          [-1, 8, 112, 112]              16
           Conv2d-13          [-1, 8, 112, 112]              72
      BatchNorm2d-14          [-1, 8, 1

In [14]:
test_data_path = 'OriginalDataset' 
test_dataset = ImageFolder(root=test_data_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [23]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Evaluate the model on validation set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = correct / total * 100
print(f'Final Validation Accuracy: {val_accuracy:.2f}%')

Final Validation Accuracy: 96.88%


In [15]:
# Evaluate the model on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total * 100
print(f'Final Test Accuracy: {test_accuracy:.2f}%')

Final Test Accuracy: 99.02%
